In [2]:
# # Model 2 - đc 0.63

# import os, sys, math, time, pickle, gc

# import numpy as np

# import pandas as pd

# from tqdm import tqdm

# import torch

# import torch.nn as nn

# import torch.optim as optim

# from torch.utils.data import Dataset, DataLoader


# class WideProteinMLP(nn.Module):
#     def __init__(self, input_dim, num_classes, hidden_dims=[2048, 4096], dropout=0.3):

#         super().__init__()

#         layers = []

#         # Input Norm: Giúp ổn định đầu vào từ Embeddings

#         layers.append(nn.LayerNorm(input_dim))

#         prev = input_dim

#         for h in hidden_dims:

#             layers.append(nn.Linear(prev, h))

#             layers.append(nn.GELU())  # Activation hiện đại

#             layers.append(nn.Dropout(dropout))

#             prev = h

#         layers.append(nn.Linear(prev, num_classes))

#         self.net = nn.Sequential(*layers)

#     def forward(self, x):

#         return self.net(x)


# # ============================================================================

# # 1. CẤU HÌNH (ĐÃ KHỚP VỚI ẢNH DATASET CỦA BẠN)

# # ============================================================================

# CONFIG = {
#     # --- Input Embeddings ---
#     "EMBED_DIR": "/kaggle/input/cafa6-embeds",
#     # --- Input Labels & Metadata (Dataset c95-cafa6) ---
#     "LABEL_DIR": "/kaggle/input/c99-cafa6",
#     "VOCAB_FILE": "vocab_C99_remove.csv",
#     "TARGET_FILE": "train_targets_C99.pkl",
#     "TRAIN_IDS": "train_ids_C99_split.npy",
#     "VAL_IDS": "val_ids_C99_split.npy",
#     "IA_FILE": "/kaggle/input/cafa-6-protein-function-prediction/IA.tsv",
#     # --- Model Params ---
#     "input_dim": 1280,
#     "hidden_dims": [2048, 4096],
#     "dropout": 0.3,
#     "batch_size": 32,
#     "lr": 2e-4,
#     "weight_decay": 1e-4,
#     "epochs": 20,
#     "device": "cuda" if torch.cuda.is_available() else "cpu",
# }


# # ============================================================================

# # 2. DATASET (SỬA LẠI ĐỂ ĐỌC ĐÚNG PATH)

# # ============================================================================


# class CAFA6Dataset(Dataset):
#     def __init__(self, ids_file, targets_file, embed_dir, num_classes):

#         self.ids = np.load(ids_file)

#         with open(targets_file, "rb") as f:

#             self.labels_dict = pickle.load(f)

#         self.num_classes = num_classes

#         # Mapping ID -> Index (Dựa vào file train_ids.txt trong folder embeds)

#         self.id_to_embed_idx = {}

#         with open(os.path.join(embed_dir, "train_ids.txt"), "r") as f:

#             for idx, line in enumerate(f):

#                 self.id_to_embed_idx[line.strip()] = idx

#         # Mmap embedding

#         self.embed_matrix = np.load(
#             os.path.join(embed_dir, "train_embeds.npy"), mmap_mode="r"
#         )

#     def __len__(self):

#         return len(self.ids)

#     def __getitem__(self, idx):

#         prot_id = self.ids[idx]

#         embed_idx = self.id_to_embed_idx.get(prot_id)

#         # Load X

#         if embed_idx is None:

#             feat = torch.zeros(1280, dtype=torch.float)

#         else:

#             feat = torch.from_numpy(self.embed_matrix[embed_idx].copy()).float()

#         # Load Y

#         target = torch.zeros(self.num_classes, dtype=torch.float)

#         indices = self.labels_dict.get(prot_id, [])

#         if len(indices) > 0:

#             target[indices] = 1.0

#         return feat, target


# # ============================================================================

# # 3. MODEL: WIDE MLP

# # ============================================================================


# class WideProteinMLP(nn.Module):
#     def __init__(self, input_dim, num_classes, hidden_dims=[2048, 4096], dropout=0.3):

#         super().__init__()

#         layers = []

#         layers.append(nn.LayerNorm(input_dim))

#         prev = input_dim

#         for h in hidden_dims:

#             layers.append(nn.Linear(prev, h))

#             layers.append(nn.GELU())

#             layers.append(nn.Dropout(dropout))

#             prev = h

#         layers.append(nn.Linear(prev, num_classes))

#         self.net = nn.Sequential(*layers)

#     def forward(self, x):

#         return self.net(x)


# # ============================================================================

# # 4. LOSS: ASL OPTIMIZED (KHỚP CÔNG THỨC ẢNH)

# # ============================================================================


# class AsymmetricLossOptimized(nn.Module):
#     def __init__(
#         self,
#         gamma_neg=4,
#         gamma_pos=0,
#         clip=0.05,
#         eps=1e-8,
#         disable_torch_grad_focal_loss=True,
#     ):

#         super(AsymmetricLossOptimized, self).__init__()

#         self.gamma_neg = gamma_neg

#         self.gamma_pos = gamma_pos

#         self.clip = clip

#         self.eps = eps

#         # Tắt gradient ở phần tính weight để tiết kiệm bộ nhớ (Mẹo tối ưu của Top Kaggler)

#         self.disable_torch_grad_focal_loss = disable_torch_grad_focal_loss

#     def forward(self, x, y):

#         # x: logits

#         # y: labels (0/1)

#         # 1. Tính xác suất P

#         x_sigmoid = torch.sigmoid(x)

#         xs_pos = x_sigmoid

#         xs_neg = 1 - x_sigmoid

#         # 2. Asymmetric Clipping (Shift xác suất âm)

#         if self.clip is not None and self.clip > 0:

#             xs_neg = (xs_neg + self.clip).clamp(max=1)

#         # 3. Tính Pt (Xác suất của nhãn đúng)

#         # pt = p nếu y=1, pt = p_shifted nếu y=0

#         pt = y * xs_pos + (1 - y) * xs_neg

#         # 4. Tính Log Pt (Base Cross Entropy)

#         # log(pt + eps)

#         log_pt = torch.log(pt.clamp(min=self.eps))

#         # 5. Tính hệ số điều chỉnh (Modulating Factor)

#         # Tắt gradient phần này để model chỉ tập trung tối ưu xác suất, không tối ưu cái weight

#         if self.disable_torch_grad_focal_loss:

#             torch.set_grad_enabled(False)

#         one_sided_gamma = self.gamma_pos * y + self.gamma_neg * (1 - y)

#         mod_factor = (1 - pt) ** one_sided_gamma

#         if self.disable_torch_grad_focal_loss:

#             torch.set_grad_enabled(True)

#         # 6. Final Loss

#         loss = -mod_factor * log_pt

#         # QUAN TRỌNG: Dùng sum() thay vì mean() cho bài toán sparse multi-label

#         return loss.sum()


# # ============================================================================

# # 5. METRIC & TRAINING LOOP

# # ============================================================================


# def calculate_fmax_subset_fast(preds, targets, ia_weights):

#     # Lọc dòng valid

#     true_sum = np.sum(targets * ia_weights, axis=1)

#     mask = true_sum > 0

#     if mask.sum() == 0:
#         return 0.0

#     p = preds[mask]

#     t = targets[mask]

#     w_sum_t = true_sum[mask]

#     w_broad = ia_weights.reshape(1, -1)

#     best_f1 = 0.0

#     thresholds = np.linspace(0.0, 1.0, 101)

#     for tau in thresholds:

#         cut = (p >= tau).astype(int)

#         tp = np.sum((cut * t) * w_broad, axis=1)

#         pred_sum = np.sum(cut * w_broad, axis=1)

#         prec = np.divide(tp, pred_sum, out=np.zeros_like(tp), where=pred_sum != 0)

#         rec = np.divide(tp, w_sum_t, out=np.zeros_like(tp), where=w_sum_t != 0)

#         f1 = 0.0

#         if (prec.mean() + rec.mean()) > 0:

#             f1 = 2 * prec.mean() * rec.mean() / (prec.mean() + rec.mean())

#         if f1 > best_f1:
#             best_f1 = f1

#     return best_f1


# def main():

#     print(
#         f"Loading vocab from {os.path.join(CONFIG['LABEL_DIR'], CONFIG['VOCAB_FILE'])}..."
#     )

#     vocab_df = pd.read_csv(os.path.join(CONFIG["LABEL_DIR"], CONFIG["VOCAB_FILE"]))

#     num_classes = len(vocab_df)

#     # Load IA Weights (Giả định file IA có sẵn, nếu ko dùng ones)

#     try:

#         ia_df = pd.read_csv(
#             CONFIG["IA_FILE"], sep="\t", names=["term", "ia"], header=None
#         )

#         ia_dict = ia_df.set_index("term")["ia"].to_dict()

#         ia_weights = np.array([ia_dict.get(t, 1.0) for t in vocab_df["term"].values])

#         print("✅ Loaded Real IA Weights.")

#     except:

#         print("⚠️ Warning: IA file not found. Using weights=1.0")

#         ia_weights = np.ones(num_classes)

#     # Datasets

#     train_ds = CAFA6Dataset(
#         os.path.join(CONFIG["LABEL_DIR"], CONFIG["TRAIN_IDS"]),
#         os.path.join(CONFIG["LABEL_DIR"], CONFIG["TARGET_FILE"]),
#         CONFIG["EMBED_DIR"],
#         num_classes,
#     )

#     val_ds = CAFA6Dataset(
#         os.path.join(CONFIG["LABEL_DIR"], CONFIG["VAL_IDS"]),
#         os.path.join(CONFIG["LABEL_DIR"], CONFIG["TARGET_FILE"]),
#         CONFIG["EMBED_DIR"],
#         num_classes,
#     )

#     train_loader = DataLoader(
#         train_ds,
#         batch_size=CONFIG["batch_size"],
#         shuffle=True,
#         num_workers=2,
#         pin_memory=True,
#     )

#     val_loader = DataLoader(
#         val_ds,
#         batch_size=CONFIG["batch_size"] * 2,
#         shuffle=False,
#         num_workers=2,
#         pin_memory=True,
#     )

#     # Model & Training

#     model = WideProteinMLP(
#         CONFIG["input_dim"], num_classes, CONFIG["hidden_dims"], CONFIG["dropout"]
#     ).to(CONFIG["device"])

#     if torch.cuda.device_count() > 1:

#         print("Using", torch.cuda.device_count(), "GPUs!")

#         model = nn.DataParallel(model)

#     model = model.to(CONFIG["device"])

#     criterion = AsymmetricLossOptimized(gamma_neg=4, gamma_pos=0.25, clip=0.05)

#     optimizer = optim.AdamW(
#         model.parameters(), lr=CONFIG["lr"], weight_decay=CONFIG["weight_decay"]
#     )

#     scheduler = optim.lr_scheduler.OneCycleLR(
#         optimizer,
#         max_lr=CONFIG["lr"],
#         steps_per_epoch=len(train_loader),
#         epochs=CONFIG["epochs"],
#     )

#     print(f"\n🚀 Start Training WideMLP ({num_classes} classes)...")

#     best_score = 0.0

#     for epoch in range(CONFIG["epochs"]):

#         model.train()

#         total_loss = 0

#         pbar = tqdm(train_loader, desc=f"Ep {epoch+1}", leave=False)

#         for x, y in pbar:

#             x, y = x.to(CONFIG["device"]), y.to(CONFIG["device"])

#             optimizer.zero_grad()

#             logits = model(x)

#             loss = criterion(logits, y)

#             loss.backward()

#             optimizer.step()

#             scheduler.step()

#             total_loss += loss.item()

#             pbar.set_postfix({"loss": f"{loss.item():.4f}"})

#         # Validation Phase

#         model.eval()

#         all_preds, all_targets = [], []

#         with torch.no_grad():

#             for x, y in val_loader:

#                 logits = model(x.to(CONFIG["device"]))

#                 all_preds.append(torch.sigmoid(logits).cpu().numpy())

#                 all_targets.append(y.numpy())

#         Y_p = np.vstack(all_preds)

#         Y_t = np.vstack(all_targets)

#         scores = {}

#         for aspect in ["MFO", "BPO", "CCO"]:

#             cols = vocab_df.index[vocab_df["aspect"] == aspect].tolist()

#             if cols:

#                 scores[aspect] = calculate_fmax_subset_fast(
#                     Y_p[:, cols], Y_t[:, cols], ia_weights[cols]
#                 )

#         avg_fmax = np.mean(list(scores.values()))

#         print(
#             f"Epoch {epoch+1}: Loss={total_loss/len(train_loader):.4f} | Val F-max={avg_fmax:.4f} {scores}"
#         )

#         if avg_fmax > best_score:

#             best_score = avg_fmax

#             torch.save(model.state_dict(), "best_model_wide.pth")

#             print("   🏆 Saved Best Model!")


In [1]:
import os
import sys
import gc
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler

# ============================================================================
# 1. CẤU HÌNH C99 (TUNED FOR TAIL)
# ============================================================================
CONFIG = {
    'EMBED_DIR': "/kaggle/input/cafa6-embeds",
    'LABEL_DIR': "/kaggle/input/c99-cafa6", # [QUAN TRỌNG] Đổi folder nếu cần
    'WORK_DIR': "/kaggle/working",
    
    # --- ĐỔI SANG FILE C99 ---
    'VOCAB_FILE': "vocab_C99_remove.csv",
    'TARGET_FILE': "train_targets_C99.pkl",
    'TRAIN_IDS': "train_ids_C99_split.npy",
    'VAL_IDS': "val_ids_C99_split.npy",
    
    'IA_FILE': "/kaggle/input/cafa-6-protein-function-prediction/IA.tsv",
    
    # --- Model Params (Giữ nguyên) ---
    'input_dim': 1280,
    'hidden_dims': [2048, 4096], # Đủ mạnh cho C99
    'dropout': 0.3,
    # num_classes sẽ tự động load từ vocab
    
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    
    # --- Training Config ---
    'batch_size': 16,    
    'lr': 2e-4,          # Giữ nguyên tốc độ chuẩn
    'weight_decay': 1e-4,
    'epochs': 25         # 20 Epoch là đủ cho Standard
}

# ============================================================================
# 2. MODEL & DATASET (GIỮ NGUYÊN BẢN GỐC)
# ============================================================================
class WideProteinMLP(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_dims=[2048, 4096], dropout=0.3):
        super().__init__()
        layers = [nn.LayerNorm(input_dim)]
        prev = input_dim
        for h in hidden_dims:
            layers += [nn.Linear(prev, h), nn.GELU(), nn.Dropout(dropout)]
            prev = h
        layers.append(nn.Linear(prev, num_classes))
        self.net = nn.Sequential(*layers)
    def forward(self, x): return self.net(x)

class CAFA6Dataset(Dataset):
    def __init__(self, ids_file, targets_file, embed_dir, num_classes):
        # Auto-detect path
        path = os.path.join(CONFIG['LABEL_DIR'], ids_file)
        if not os.path.exists(path): path = os.path.join(CONFIG['WORK_DIR'], ids_file)
        self.ids = np.load(path)
        
        t_path = os.path.join(CONFIG['LABEL_DIR'], targets_file)
        if not os.path.exists(t_path): t_path = os.path.join(CONFIG['WORK_DIR'], targets_file)
        with open(t_path, 'rb') as f: self.labels_dict = pickle.load(f)
            
        self.num_classes = num_classes
        self.id_to_embed_idx = {}
        with open(os.path.join(embed_dir, "train_ids.txt"), 'r') as f:
            for idx, line in enumerate(f): self.id_to_embed_idx[line.strip()] = idx
        self.embed_matrix = np.load(os.path.join(embed_dir, "train_embeds.npy"), mmap_mode='r')

    def __len__(self): return len(self.ids)
    def __getitem__(self, idx):
        prot_id = self.ids[idx]
        embed_idx = self.id_to_embed_idx.get(prot_id)
        feat = torch.from_numpy(self.embed_matrix[embed_idx].copy()).float() if embed_idx is not None else torch.zeros(1280)
        target = torch.zeros(self.num_classes, dtype=torch.float)
        indices = self.labels_dict.get(prot_id, [])
        if len(indices) > 0: target[indices] = 1.0
        return feat, target

# ============================================================================
# 3. LOSS FUNCTION (ASL)
# ============================================================================
class AsymmetricLossOptimized(nn.Module):
    def __init__(self, gamma_neg=4, gamma_pos=0, clip=0.05, eps=1e-8):
        super().__init__()
        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.eps = eps

    def forward(self, x, y):
        x_sigmoid = torch.sigmoid(x)
        xs_pos = x_sigmoid
        xs_neg = 1 - x_sigmoid
        if self.clip > 0: xs_neg = (xs_neg + self.clip).clamp(max=1)
        
        pt = y * xs_pos + (1 - y) * xs_neg
        log_pt = torch.log(pt.clamp(min=self.eps))
        
        pos_weight = (1 - xs_pos) ** self.gamma_pos
        neg_weight = (1 - xs_neg) ** self.gamma_neg
        
        loss = - (pos_weight * log_pt * y + neg_weight * log_pt * (1-y))
        return loss.sum()

# ============================================================================
# 4. METRIC & MAIN (DÙNG HÀM METRIC CHUẨN CỦA BẠN)
# ============================================================================
def calculate_fmax_subset(preds, targets, ia_weights):
    w = ia_weights.reshape(1, -1)
    true_sum = np.sum(targets * w, axis=1)
    valid_mask = true_sum > 0
    if valid_mask.sum() == 0: return 0.0
    
    p_sub = preds[valid_mask]
    t_sub = targets[valid_mask]
    w_sub = w 
    w_true_sub = true_sum[valid_mask]
    
    best_f1 = 0.0
    # Quét 51 ngưỡng cho nhanh
    thresholds = np.linspace(0.0, 1.0, 51) 
    
    for tau in thresholds:
        cut = (p_sub >= tau).astype(int)
        tp = np.sum((cut * t_sub) * w_sub, axis=1)
        pred_sum = np.sum(cut * w_sub, axis=1)
        
        prec = np.divide(tp, pred_sum, out=np.zeros_like(tp), where=pred_sum!=0)
        rec = np.divide(tp, w_true_sub, out=np.zeros_like(tp), where=w_true_sub!=0)
        
        avg_p = np.mean(prec)
        avg_r = np.mean(rec)
        
        if (avg_p + avg_r) > 0:
            f1 = 2 * avg_p * avg_r / (avg_p + avg_r)
        else:
            f1 = 0.0
            
        if f1 > best_f1: best_f1 = f1
            
    return best_f1

def validate_detailed(model, loader, vocab_df, ia_weights, device):
    model.eval()
    all_preds, all_targets = [], []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            with autocast():
                logits = model(x)
            all_preds.append(torch.sigmoid(logits).cpu().numpy())
            all_targets.append(y.numpy())
    Y_p = np.vstack(all_preds)
    Y_t = np.vstack(all_targets)
    
    scores = {}
    # Lưu ý: C99 có nhiều nhãn hơn, nên việc loop aspect vẫn đúng
    for aspect in ['MFO', 'BPO', 'CCO']:
        col_indices = vocab_df.index[vocab_df['aspect'] == aspect].tolist()
        if not col_indices: continue
        scores[aspect] = calculate_fmax_subset(Y_p[:, col_indices], Y_t[:, col_indices], ia_weights[col_indices])
    
    avg_fmax = np.mean(list(scores.values()))
    return avg_fmax, scores

def main():
    print(f"🚀 START TRAINING C99 STANDARD (WIDE MLP)...")
    
    # 1. Load Vocab C99
    path = os.path.join(CONFIG['LABEL_DIR'], CONFIG['VOCAB_FILE'])
    if not os.path.exists(path): path = os.path.join(CONFIG['WORK_DIR'], CONFIG['VOCAB_FILE'])
    vocab_df = pd.read_csv(path)
    num_classes = len(vocab_df)
    print(f"   Num Classes: {num_classes}")
    
    # Load IA Weights (Map theo Vocab C99)
    try:
        ia_df = pd.read_csv(CONFIG['IA_FILE'], sep='\t', names=['term', 'ia'], header=None)
        ia_map = dict(zip(ia_df.term, ia_df.ia))
        ia_weights = np.array([ia_map.get(t, 1.0) for t in vocab_df.term.values])
    except: ia_weights = np.ones(num_classes)
    
    # 2. Loaders
    train_ds = CAFA6Dataset(CONFIG['TRAIN_IDS'], CONFIG['TARGET_FILE'], CONFIG['EMBED_DIR'], num_classes)
    val_ds = CAFA6Dataset(CONFIG['VAL_IDS'], CONFIG['TARGET_FILE'], CONFIG['EMBED_DIR'], num_classes)
    train_loader = DataLoader(train_ds, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=2)
    val_loader = DataLoader(val_ds, batch_size=CONFIG['batch_size']*2, shuffle=False, num_workers=2)
    
    # 3. Model
    model = WideProteinMLP(CONFIG['input_dim'], num_classes, CONFIG['hidden_dims'], CONFIG['dropout']).to(CONFIG['device'])
    if torch.cuda.device_count() > 1: model = nn.DataParallel(model)
    
    # 4. Config Tối ưu cho C99 (GammaPos=1.0)
    # [QUAN TRỌNG]: Tăng gamma_pos lên 1.0 để bắt nhãn hiếm (C99 nhiều nhãn hiếm)
    criterion = AsymmetricLossOptimized(gamma_neg=2.0, gamma_pos=2.0, clip=0.05)
    
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG['lr'], weight_decay=CONFIG['weight_decay'])
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=CONFIG['lr'], steps_per_epoch=len(train_loader), epochs=CONFIG['epochs'])
    scaler = GradScaler()

    best_score = 0.0
    
    # 5. Training Loop
    for epoch in range(CONFIG['epochs']):
        model.train()
        loss_sum = 0
        pbar = tqdm(train_loader, desc=f"Ep {epoch+1}", leave=False)
        
        for x, y in pbar:
            x, y = x.to(CONFIG['device']), y.to(CONFIG['device'])
            optimizer.zero_grad()
            with autocast():
                logits = model(x)
                loss = criterion(logits, y)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            loss_sum += loss.item()
            pbar.set_postfix({'loss': f"{loss.item():.4f}"})
            
        # Validate
        val_fmax, val_details = validate_detailed(model, val_loader, vocab_df, ia_weights, CONFIG['device'])
        print(f"Epoch {epoch+1}: Loss={loss_sum/len(train_loader):.4f} | Val F-max={val_fmax:.4f} {val_details}")
        
        if val_fmax > best_score:
            best_score = val_fmax
            torch.save(model.state_dict(), "best_model_c99_extreme.pth")
            print("   🏆 Saved Best C99 Model!")

    print(f"\n✅ DONE! Best F-max: {best_score:.4f}")

if __name__ == "__main__":
    main()

🚀 START TRAINING C99 STANDARD (WIDE MLP)...
   Num Classes: 15582


/tmp/ipykernel_47/2595228453.py:208: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Ep 1:   0%|          | 0/2318 [00:00<?, ?it/s]/tmp/ipykernel_47/2595228453.py:221: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
/tmp/ipykernel_47/2595228453.py:158: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Pleas

Epoch 1: Loss=1349.6962 | Val F-max=0.3839 {'MFO': 0.4326490736805841, 'BPO': 0.28492136867520496, 'CCO': 0.4339804102579939}
   🏆 Saved Best C99 Model!


Epoch 2: Loss=930.5789 | Val F-max=0.4441 {'MFO': 0.49905937145915946, 'BPO': 0.3285555580871629, 'CCO': 0.5046982517380255}
   🏆 Saved Best C99 Model!


Epoch 3: Loss=862.5929 | Val F-max=0.4717 {'MFO': 0.5263946454431395, 'BPO': 0.351633652256163, 'CCO': 0.5369894031558298}
   🏆 Saved Best C99 Model!


...

Epoch 16: Loss=562.3977 | Val F-max=0.5650 {'MFO': 0.6457414387138274, 'BPO': 0.43982078554072934, 'CCO': 0.6094999180016677}
   🏆 Saved Best C99 Model!


Epoch 17: Loss=551.7101 | Val F-max=0.5655 {'MFO': 0.6453957461929413, 'BPO': 0.44214246873513435, 'CCO': 0.6088716858662978}
   🏆 Saved Best C99 Model!


Epoch 18: Loss=528.6237 | Val F-max=0.5660 {'MFO': 0.6465341191956316, 'BPO': 0.44229320178259657, 'CCO': 0.6092493178172143}
   🏆 Saved Best C99 Model!


Epoch 19: Loss=521.6768 | Val F-max=0.5667 {'MFO': 0.647140731363098, 'BPO': 0.44288150943292076, 'CCO': 0.6101694024449723}
   🏆 Saved Best C99 Model!


Epoch 20: Loss=519.3324 | Val F-max=0.5668 {'MFO': 0.6473101011950865, 'BPO': 0.44370479902528787, 'CCO': 0.6095005918401467}
   🏆 Saved Best C99 Model!

✅ DONE! Best F-max: 0.5668


In [5]:
# !pip install obonet networkx --quiet

# import obonet
# import networkx as nx
# import pandas as pd
# import numpy as np
# import pickle
# from collections import deque
# import os

# CONFIG = {
#     'OBO_FILE': "/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo",
#     'VOCAB_FILE': "/kaggle/input/c95-cafa6/vocab_C95_remove.csv", 
#     'TRAIN_TARGETS': "/kaggle/input/c95-cafa6/train_targets_C95.pkl", # Cần cho tính IC
#     'OUTPUT_PKL': "hierarchy_metadata_v2.pkl"
# }

# def build_training_metadata_full():
#     print("🚀 Building Training Metadata (Depth, IC & HMC Map)...")
    
#     # 1. Load Graph & Vocab C95
#     graph = obonet.read_obo(CONFIG['OBO_FILE'])
#     vocab_df = pd.read_csv(CONFIG['VOCAB_FILE'])
#     vocab_terms = vocab_df['term'].tolist()
#     go_to_idx = {go: i for i, go in enumerate(vocab_terms)}
#     n_classes = len(vocab_terms)
    
#     # ----------------------------------------------------
#     # 2. TÍNH DEPTH (BFS) - Logic của bạn
#     # ----------------------------------------------------
#     roots = ["GO:0008150", "GO:0003674", "GO:0005575"]
#     depth_dict = {node: float('inf') for node in graph.nodes()}
#     queue = deque()
    
#     for root in roots:
#         if root in graph:
#             depth_dict[root] = 0
#             queue.append(root)
            
#     while queue:
#         node = queue.popleft()
#         d = depth_dict[node]
#         # Sử dụng 'is_a' và 'part_of' edges (chỉ cần lấy edge predecessors)
#         for child in graph.predecessors(node):
#             if depth_dict[child] > d + 1:
#                 depth_dict[child] = d + 1
#                 queue.append(child)
                
#     # Map Depth vào Vocab C95
#     depth_arr = np.array([depth_dict.get(t, 0) for t in vocab_terms], dtype=np.float32)
#     depth_arr[depth_arr == float('inf')] = 0.0
    
#     # Normalize
#     max_d = depth_arr.max()
#     depth_norm = depth_arr / max_d if max_d > 0 else depth_arr
#     print(f"  ✅ Depth Done. Max Depth: {max_d}")
    
#     # ----------------------------------------------------
#     # 3. TÍNH INFORMATION CONTENT (IC)
#     # ----------------------------------------------------
#     # Load targets để tính tần suất
#     with open(CONFIG['TRAIN_TARGETS'], 'rb') as f:
#         labels_dict = pickle.load(f)
    
#     # Tính tần suất (Count)
#     term_counts = np.zeros(n_classes, dtype=np.int32)
#     total_proteins = len(labels_dict)

#     # Tổng hợp nhãn từ dictionary
#     for pid, indices in labels_dict.items():
#         # Chỉ cần đảm bảo index không vượt quá n_classes (đã được lọc C95)
#         for idx in indices:
#             if idx < n_classes:
#                 term_counts[idx] += 1
    
#     # Công thức IC: IC = -log(P(t))
#     term_prob = term_counts / total_proteins
#     # Thêm epsilon nhỏ để tránh log(0)
#     term_prob = np.clip(term_prob, 1e-8, 1.0)
#     ic_arr = -np.log(term_prob)
    
#     # Normalize IC (Optional, nhưng an toàn hơn)
#     max_ic = ic_arr.max()
#     ic_norm = ic_arr / max_ic if max_ic > 0 else ic_arr
#     print(f"  ✅ IC Done. Max IC: {max_ic:.2f}")

#     # ----------------------------------------------------
#     # 4. CHILD-TO-PARENT INDEX MAP (Cho HMC Loss)
#     # ----------------------------------------------------
#     child_to_parent_idx = {}
    
#     # Chỉ duyệt qua các GO Term CÓ trong vocab C95
#     for term_go in tqdm(vocab_terms, desc="Building Parent Map"):
#         if term_go not in graph: continue
        
#         child_idx = go_to_idx[term_go]
#         parent_indices = []
        
#         # predecessors = parents trong graph
#         for parent_go in graph.successors(term_go):
#             if parent_go in go_to_idx:
#                 parent_indices.append(go_to_idx[parent_go])
        
#         if parent_indices:
#             child_to_parent_idx[child_idx] = parent_indices

#     print(f"  ✅ HMC Map Done. Total nodes mapped: {len(child_to_parent_idx):,}")
    
#     # 5. Save
#     metadata = {
#         'depth_norm': depth_norm,
#         'ic_norm': ic_norm,
#         'child_to_parent_idx': child_to_parent_idx
#     }
#     with open(CONFIG['OUTPUT_PKL'], 'wb') as f:
#         pickle.dump(metadata, f)
        
#     print(f"\n✅ DONE! Saved all metadata to {CONFIG['OUTPUT_PKL']}")

# if __name__ == "__main__":
#     # Đảm bảo file TRAIN_TARGETS đã được mount hoặc tạo ra
#     if not os.path.exists(CONFIG['TRAIN_TARGETS']):
#         print(f"🚨 ERROR: Missing {CONFIG['TRAIN_TARGETS']}. Cannot calculate IC.")
#     else:
#         build_training_metadata_full()

🚀 Building Training Metadata (Depth, IC & HMC Map)...
  ✅ Depth Done. Max Depth: 11.0
  ✅ IC Done. Max IC: 7.66


Building Parent Map: 100%|██████████| 6413/6413 [00:00<00:00, 459748.94it/s]

  ✅ HMC Map Done. Total nodes mapped: 6,375

✅ DONE! Saved all metadata to hierarchy_metadata_v2.pkl


In [17]:
# !pip install obonet networkx --quiet

# import obonet
# import networkx as nx
# import pandas as pd
# import numpy as np
# import pickle
# from collections import deque
# import os

# CONFIG = {
#     'OBO_FILE': "/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo",
#     'VOCAB_FILE': "/kaggle/input/c99-cafa6/vocab_C99_remove.csv", 
#     'TRAIN_TARGETS': "/kaggle/input/c99-cafa6/train_targets_C99.pkl", # Cần cho tính IC
#     'OUTPUT_PKL': "hierarchy_metadata_C99.pkl"
# }

# def build_training_metadata_full():
#     print("🚀 Building Training Metadata (Depth, IC & HMC Map)...")
    
#     # 1. Load Graph & Vocab C95
#     graph = obonet.read_obo(CONFIG['OBO_FILE'])
#     vocab_df = pd.read_csv(CONFIG['VOCAB_FILE'])
#     vocab_terms = vocab_df['term'].tolist()
#     go_to_idx = {go: i for i, go in enumerate(vocab_terms)}
#     n_classes = len(vocab_terms)
    
#     # ----------------------------------------------------
#     # 2. TÍNH DEPTH (BFS) - Logic của bạn
#     # ----------------------------------------------------
#     roots = ["GO:0008150", "GO:0003674", "GO:0005575"]
#     depth_dict = {node: float('inf') for node in graph.nodes()}
#     queue = deque()
    
#     for root in roots:
#         if root in graph:
#             depth_dict[root] = 0
#             queue.append(root)
            
#     while queue:
#         node = queue.popleft()
#         d = depth_dict[node]
#         # Sử dụng 'is_a' và 'part_of' edges (chỉ cần lấy edge predecessors)
#         for child in graph.predecessors(node):
#             if depth_dict[child] > d + 1:
#                 depth_dict[child] = d + 1
#                 queue.append(child)
                
#     # Map Depth vào Vocab C95
#     depth_arr = np.array([depth_dict.get(t, 0) for t in vocab_terms], dtype=np.float32)
#     depth_arr[depth_arr == float('inf')] = 0.0
    
#     # Normalize
#     max_d = depth_arr.max()
#     depth_norm = depth_arr / max_d if max_d > 0 else depth_arr
#     print(f"  ✅ Depth Done. Max Depth: {max_d}")
    
#     # ----------------------------------------------------
#     # 3. TÍNH INFORMATION CONTENT (IC)
#     # ----------------------------------------------------
#     # Load targets để tính tần suất
#     with open(CONFIG['TRAIN_TARGETS'], 'rb') as f:
#         labels_dict = pickle.load(f)
    
#     # Tính tần suất (Count)
#     term_counts = np.zeros(n_classes, dtype=np.int32)
#     total_proteins = len(labels_dict)

#     # Tổng hợp nhãn từ dictionary
#     for pid, indices in labels_dict.items():
#         # Chỉ cần đảm bảo index không vượt quá n_classes (đã được lọc C95)
#         for idx in indices:
#             if idx < n_classes:
#                 term_counts[idx] += 1
    
#     # Công thức IC: IC = -log(P(t))
#     term_prob = term_counts / total_proteins
#     # Thêm epsilon nhỏ để tránh log(0)
#     term_prob = np.clip(term_prob, 1e-8, 1.0)
#     ic_arr = -np.log(term_prob)
    
#     # Normalize IC (Optional, nhưng an toàn hơn)
#     max_ic = ic_arr.max()
#     ic_norm = ic_arr / max_ic if max_ic > 0 else ic_arr
#     print(f"  ✅ IC Done. Max IC: {max_ic:.2f}")

#     # ----------------------------------------------------
#     # 4. CHILD-TO-PARENT INDEX MAP (Cho HMC Loss)
#     # ----------------------------------------------------
#     child_to_parent_idx = {}
    
#     # Chỉ duyệt qua các GO Term CÓ trong vocab C95
#     for term_go in tqdm(vocab_terms, desc="Building Parent Map"):
#         if term_go not in graph: continue
        
#         child_idx = go_to_idx[term_go]
#         parent_indices = []
        
#         # predecessors = parents trong graph
#         for parent_go in graph.successors(term_go):
#             if parent_go in go_to_idx:
#                 parent_indices.append(go_to_idx[parent_go])
        
#         if parent_indices:
#             child_to_parent_idx[child_idx] = parent_indices

#     print(f"  ✅ HMC Map Done. Total nodes mapped: {len(child_to_parent_idx):,}")
    
#     # 5. Save
#     metadata = {
#         'depth_norm': depth_norm,
#         'ic_norm': ic_norm,
#         'child_to_parent_idx': child_to_parent_idx
#     }
#     with open(CONFIG['OUTPUT_PKL'], 'wb') as f:
#         pickle.dump(metadata, f)
        
#     print(f"\n✅ DONE! Saved all metadata to {CONFIG['OUTPUT_PKL']}")

# if __name__ == "__main__":
#     # Đảm bảo file TRAIN_TARGETS đã được mount hoặc tạo ra
#     if not os.path.exists(CONFIG['TRAIN_TARGETS']):
#         print(f"🚨 ERROR: Missing {CONFIG['TRAIN_TARGETS']}. Cannot calculate IC.")
#     else:
#         build_training_metadata_full()

🚀 Building Training Metadata (Depth, IC & HMC Map)...
  ✅ Depth Done. Max Depth: 11.0
  ✅ IC Done. Max IC: 9.37


Building Parent Map: 100%|██████████| 15582/15582 [00:00<00:00, 415075.07it/s]

  ✅ HMC Map Done. Total nodes mapped: 15,535

✅ DONE! Saved all metadata to hierarchy_metadata_C99.pkl


In [ ]:
# import os
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# from torch.cuda.amp import autocast, GradScaler
# import numpy as np
# import pandas as pd
# import pickle
# from tqdm import tqdm
# from collections import OrderedDict

# # ============================================================================
# # 1. CẤU HÌNH C99 (FIX LỖI PATH)
# # ============================================================================
# CONFIG = {
#     # --- PATHS ---
#     'EMBED_DIR': "/kaggle/input/cafa6-embeds",
    
#     # [FIX 1] Trỏ đúng vào thư mục chứa dữ liệu C99
#     'LABEL_DIR': "/kaggle/input/c99-cafa6", 
#     'WORK_DIR': "/kaggle/working",
    
#     # Files C99
#     'VOCAB_FILE': "vocab_C99_remove.csv",       
#     'TARGET_FILE': "train_targets_C99.pkl",     
#     'TRAIN_IDS': "train_ids_C99_split.npy",     
#     'VAL_IDS': "val_ids_C99_split.npy",         
#     'METADATA_PKL': "hierarchy_metadata_C99.pkl", 
#     'IA_FILE': "/kaggle/input/cafa-6-protein-function-prediction/IA.tsv",
    
#     # --- MODEL SURGERY ---
#     'PRETRAINED_MODEL': "model_v3_hmc_warmup.pth", 
#     'OLD_VOCAB_FILE': "/kaggle/input/c95-cafa6/vocab_C95_remove.csv",
    
#     # --- MODEL SPECS ---
#     'input_dim': 1280,
#     'hidden_dims': [2048, 4096],
#     'dropout': 0.3,
#     'num_classes': 15582,  
    
#     # --- TRAINING ---
#     'device': 'cuda',
#     'batch_size': 32,      
#     'lr': 1e-4,             
#     'weight_decay': 1e-4,
#     'epochs': 20,           
#     'hmc_lambda': 0.01,    
#     'warmup_epochs': 3      
# }

# # (Giữ nguyên CAFA6Dataset, WideProteinMLP)
# class CAFA6Dataset(Dataset):
#     def __init__(self, ids_file, targets_file, embed_dir, num_classes):
#         path = os.path.join(CONFIG['LABEL_DIR'], ids_file)
#         if not os.path.exists(path): path = os.path.join(CONFIG['WORK_DIR'], ids_file)
#         self.ids = np.load(path)
        
#         t_path = os.path.join(CONFIG['LABEL_DIR'], targets_file)
#         if not os.path.exists(t_path): t_path = os.path.join(CONFIG['WORK_DIR'], targets_file)
#         with open(t_path, 'rb') as f: self.labels_dict = pickle.load(f)
            
#         self.num_classes = num_classes
#         self.id_to_embed_idx = {}
#         with open(os.path.join(embed_dir, "train_ids.txt"), 'r') as f:
#             for idx, line in enumerate(f): self.id_to_embed_idx[line.strip()] = idx
#         self.embed_matrix = np.load(os.path.join(embed_dir, "train_embeds.npy"), mmap_mode='r')

#     def __len__(self): return len(self.ids)
#     def __getitem__(self, idx):
#         prot_id = self.ids[idx]
#         embed_idx = self.id_to_embed_idx.get(prot_id)
#         feat = torch.from_numpy(self.embed_matrix[embed_idx].copy()).float() if embed_idx is not None else torch.zeros(1280)
#         target = torch.zeros(self.num_classes, dtype=torch.float)
#         indices = self.labels_dict.get(prot_id, [])
#         if len(indices) > 0: target[indices] = 1.0
#         return feat, target

# class WideProteinMLP(nn.Module):
#     def __init__(self, input_dim, num_classes, hidden_dims=[2048, 4096], dropout=0.3):
#         super().__init__()
#         layers = [nn.LayerNorm(input_dim)]
#         prev = input_dim
#         for h in hidden_dims:
#             layers += [nn.Linear(prev, h), nn.GELU(), nn.Dropout(dropout)]
#             prev = h
#         layers.append(nn.Linear(prev, num_classes))
#         self.net = nn.Sequential(*layers)
#     def forward(self, x): return self.net(x)

# # ============================================================================
# # 2. MODEL SURGERY (FIX RỦI RO)
# # ============================================================================
# def load_pretrained_and_expand(model, pretrained_path, old_vocab_path, new_vocab_path):
#     print("🏥 Starting Model Surgery (C95 -> C99)...")
    
#     ckpt = torch.load(pretrained_path, map_location='cpu')
#     old_state = ckpt['model_state'] if isinstance(ckpt, dict) and 'model_state' in ckpt else ckpt
    
#     clean_state = OrderedDict()
#     for k, v in old_state.items(): clean_state[k.replace("module.", "")] = v
    
#     old_vocab = pd.read_csv(old_vocab_path)['term'].tolist()
#     new_vocab = pd.read_csv(new_vocab_path)['term'].tolist()
#     old_term_to_idx = {t: i for i, t in enumerate(old_vocab)}
    
#     classifier_weight_key = None
#     for k, v in clean_state.items():
#         # [FIX 2] Kiểm tra kỹ hơn (ndim=2 và shape match)
#         if v.ndim == 2 and v.shape[0] == len(old_vocab):
#             classifier_weight_key = k
#             break
            
#     if classifier_weight_key is None:
#         print("❌ ERROR: Không tìm thấy layer classifier cũ.")
#         return model
        
#     classifier_bias_key = classifier_weight_key.replace("weight", "bias")
#     print(f"   - Found classifier layers: {classifier_weight_key}")
    
#     new_state = model.state_dict()
#     new_weight = new_state[classifier_weight_key].clone()
#     new_bias = new_state[classifier_bias_key].clone()
    
#     old_weight = clean_state[classifier_weight_key]
#     old_bias = clean_state[classifier_bias_key]
    
#     print("   - Copying weights...")
#     copied_count = 0
#     for new_idx, term in enumerate(new_vocab):
#         if term in old_term_to_idx:
#             old_idx = old_term_to_idx[term]
#             new_weight[new_idx] = old_weight[old_idx]
#             new_bias[new_idx] = old_bias[old_idx]
#             copied_count += 1
            
#     print(f"   - Copied {copied_count}/{len(new_vocab)} terms from C95.")
    
#     # Update state dict mới
#     final_state = OrderedDict()
#     for k in new_state: # Duyệt trên cấu trúc MỚI
#         if k == classifier_weight_key:
#             final_state[k] = new_weight
#         elif k == classifier_bias_key:
#             final_state[k] = new_bias
#         elif k in clean_state:
#             final_state[k] = clean_state[k]
#         else:
#             # Layer mới (ví dụ dropout thêm vào) -> giữ random init
#             final_state[k] = new_state[k]

#     # [FIX 2] Load Strict=False và kiểm tra log
#     missing, unexpected = model.load_state_dict(final_state, strict=False)
#     print(f"   - Missing keys: {len(missing)}")
#     print(f"   - Unexpected keys: {len(unexpected)}")
    
#     print("✅ Surgery Complete.")
#     return model

# # ============================================================================
# # 3. LOSS FUNCTION (FIXED: SUM SCALE & SAFE HMC)
# # ============================================================================
# class HybridLossC99(nn.Module):
#     def __init__(self, ia_weights, hierarchy_map, gamma_neg=4, gamma_pos=0, hmc_lambda=0.01, clip=0.05,):
#         super().__init__()
#         self.gamma_neg = gamma_neg
#         self.gamma_pos = gamma_pos
#         # 🚨 THÊM DÒNG NÀY (Clip ban đầu là 0.05 theo ASL chuẩn)
#         self.clip = clip
        
#         self.hmc_lambda = hmc_lambda
        
#         # IA Weights
#         ia_tensor = torch.tensor(ia_weights, dtype=torch.float32)
#         ia_tensor = torch.clamp(ia_tensor, 0.5, 3.0)
#         # Normalize mean=1
#         ia_tensor = ia_tensor / ia_tensor.mean()
#         self.register_buffer('ia_weights', ia_tensor)
        
#         # HMC Edges
#         edges = []
#         if hierarchy_map:
#             for child, parents in hierarchy_map.items():
#                 for p in parents: edges.append([int(child), int(p)])
#         if edges:
#             self.register_buffer('edge_index', torch.tensor(edges, dtype=torch.long).t())
#         else:
#             self.edge_index = None

#     def forward(self, x, y):
#         x_sig = torch.sigmoid(x)
#         xs_pos = x_sig
#         xs_neg = 1 - x_sig
        
#         # Kẹp giá trị an toàn
#         if self.clip > 0: xs_neg = (xs_neg + self.clip).clamp(max=1)
            
#         pt = y * xs_pos + (1 - y) * xs_neg
#         log_pt = torch.log(pt.clamp(min=1e-8))
        
#         pos_W = (1 - xs_pos) ** self.gamma_pos
#         neg_W = (1 - xs_neg) ** self.gamma_neg
        
#         base_loss = - (pos_W * log_pt * y + neg_W * log_pt * (1-y))
        
#         # IA Weighted
#         w_loss = base_loss * (y * self.ia_weights + (1-y))
        
#         # [FIX] Dùng .sum() / batch_size để về scale cũ (dễ tune hơn mean cực nhỏ)
#         # Kết quả Loss sẽ tầm 1000-2000
#         asl_loss = w_loss.sum() / x.size(0)
        
#         # HMC Loss
#         if self.hmc_lambda > 0 and self.edge_index is not None:
#             c_probs = x_sig[:, self.edge_index[0]]
#             p_probs = x_sig[:, self.edge_index[1]]
#             diff = torch.relu(c_probs - p_probs)
            
#             # [FIX] HMC cũng scale theo batch
#             hmc_val = diff.sum() / x.size(0)
            
#             # [FIX] Safety Cap cho HMC (không cho nó quá lớn làm nổ gradient)
#             # Nếu HMC lớn quá, kẹp nó lại
#             # hmc_val = torch.clamp(hmc_val, max=1000.0) 
            
#             return asl_loss + (self.hmc_lambda * hmc_val)
            
#         return asl_loss

# # ============================================================================
# # 4. UTILS VALIDATION (BỔ SUNG QUAN TRỌNG)
# # ============================================================================
# def calculate_fmax_fast_c99(preds, targets, ia_weights):
#     # Lọc dòng valid
#     w = ia_weights.reshape(1, -1)
#     true_sum = np.sum(targets * w, axis=1)
#     valid_mask = true_sum > 0
#     if valid_mask.sum() == 0: return 0.0
    
#     p = preds[valid_mask]
#     t = targets[valid_mask]
#     w_sub = w 
#     w_true_sub = true_sum[valid_mask]
    
#     best_f1 = 0.0
#     # Thử quét ngưỡng thấp hơn: 0.01 đến 0.5
#     thresholds = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5] 
    
#     for tau in thresholds:
#         cut = (p >= tau).astype(int)
#         tp = np.sum((cut * t) * w_sub, axis=1)
#         pred_sum = np.sum(cut * w_sub, axis=1)
        
#         prec = np.divide(tp, pred_sum, out=np.zeros_like(tp), where=pred_sum!=0)
#         rec = np.divide(tp, w_true_sub, out=np.zeros_like(tp), where=w_true_sub!=0)
        
#         # Micro hay Macro không quan trọng lúc debug, miễn là nó > 0
#         f1_arr = np.divide(2*prec*rec, prec+rec, out=np.zeros_like(prec), where=(prec+rec)!=0)
#         f1 = np.mean(f1_arr)
#         if f1 > best_f1: best_f1 = f1
            
#     return best_f1

# # ============================================================================
# # 5. TRAINING LOOP
# # ============================================================================
# def train_c99_finetune_fixed():
#     print("🚀 STARTING C99 FINE-TUNING (FIXED)...")
    
#     # 1. Resources
#     print("   Loading Vocab C99...")
#     path = os.path.join(CONFIG['LABEL_DIR'], CONFIG['VOCAB_FILE'])
#     if not os.path.exists(path): path = os.path.join(CONFIG['WORK_DIR'], CONFIG['VOCAB_FILE'])
#     vocab_df = pd.read_csv(path)
#     CONFIG['num_classes'] = len(vocab_df)
    
#     try:
#         ia_df = pd.read_csv(CONFIG['IA_FILE'], sep='\t', names=['t','w'], header=None)
#         ia_map = dict(zip(ia_df.t, ia_df.w))
#         ia_weights = np.array([ia_map.get(t, 1.0) for t in vocab_df.term.values])
#     except: ia_weights = np.ones(CONFIG['num_classes'])
    
#     # Metadata (HMC Map)
#     path = os.path.join(CONFIG['WORK_DIR'], CONFIG['METADATA_PKL'])
#     if not os.path.exists(path): path = os.path.join(CONFIG['LABEL_DIR'], CONFIG['METADATA_PKL'])
#     with open(path, 'rb') as f: meta = pickle.load(f)
    
#     # [FIX 4] Lấy đúng key cho HMC (kiểm tra lại file metadata của bạn dùng key nào)
#     # Thường là 'child_to_parent' hoặc 'child_to_parent_idx'
#     if 'child_to_parent' in meta:
#         parent_map = meta['child_to_parent']
#     else:
#         parent_map = meta.get('child_to_parent_idx', {})

#     # 2. Datasets
#     train_ds = CAFA6Dataset(CONFIG['TRAIN_IDS'], CONFIG['TARGET_FILE'], CONFIG['EMBED_DIR'], CONFIG['num_classes'])
#     val_ds = CAFA6Dataset(CONFIG['VAL_IDS'], CONFIG['TARGET_FILE'], CONFIG['EMBED_DIR'], CONFIG['num_classes'])
#     train_loader = DataLoader(train_ds, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=2)
#     # Val loader (Sample nhỏ để check nhanh)
#     val_loader = DataLoader(val_ds, batch_size=CONFIG['batch_size']*2, shuffle=False, num_workers=2)
    
#     # 3. Model Surgery
#     model = WideProteinMLP(CONFIG['input_dim'], CONFIG['num_classes']).to(CONFIG['device'])
    
#     if os.path.exists(CONFIG['PRETRAINED_MODEL']):
#         old_vocab_path = os.path.join("/kaggle/input/c95-cafa6", "vocab_C95_remove.csv") # Path cứng C95
#         new_vocab_path = os.path.join(CONFIG['LABEL_DIR'], CONFIG['VOCAB_FILE'])
#         model = load_pretrained_and_expand(model, CONFIG['PRETRAINED_MODEL'], old_vocab_path, new_vocab_path)
#     else:
#         print("⚠️ Warning: Train from Scratch!")
        
#     if torch.cuda.device_count() > 1: model = nn.DataParallel(model)
    
#     # 4. Config
#     # [FIX] Config Loss & Optim
#     criterion = HybridLossC99(ia_weights, parent_map, gamma_neg=4, gamma_pos=0.5, hmc_lambda=0.0).to(CONFIG['device'])
    
#     # [FIX] Quay về LR chuẩn 2e-4 (vì dùng sum scale)
#     optimizer = optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-4)
#     scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=2e-4, steps_per_epoch=len(train_loader), epochs=CONFIG['epochs'])
#     scaler = torch.amp.GradScaler('cuda')

#     best_score = 0.0

#     # 5. Train Loop
#     for epoch in range(CONFIG['epochs']):
#         model.train()
        
#         if epoch < CONFIG['warmup_epochs']:
#             criterion.hmc_lambda = 0.0
#             print(f"   🔥 Epoch {epoch+1}: HMC OFF")
#         else:
#             criterion.hmc_lambda = CONFIG['hmc_lambda']
#             print(f"   🔒 Epoch {epoch+1}: HMC ON")
            
#         loss_sum = 0
#         pbar = tqdm(train_loader, desc=f"Ep {epoch+1}", leave=False)
        
#         for x, y in pbar:
#             x, y = x.to(CONFIG['device']), y.to(CONFIG['device'])
#             optimizer.zero_grad()
#             with autocast():
#                 logits = model(x)
#                 loss = criterion(logits, y)
            
#             # Clipping 1.0 bắt buộc
#             scaler.scale(loss).backward()
#             scaler.unscale_(optimizer)
#             torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
#             scaler.step(optimizer)
#             scaler.update()
#             scheduler.step()
#             loss_sum += loss.item()
#             pbar.set_postfix({'loss': f"{loss.item():.4f}"})
            
#         # Validation
#         if (epoch + 1) % 2 == 0:
#             model.eval()
#             preds, targets = [], []
#             with torch.no_grad():
#                 for i, (x, y) in enumerate(val_loader):
#                     if i > 50: break 
#                     logits = model(x.to(CONFIG['device']))
#                     preds.append(torch.sigmoid(logits).cpu().numpy())
#                     targets.append(y.numpy())
#             Y_p = np.vstack(preds)
#             Y_t = np.vstack(targets)
#             # Dùng hàm F-max chuẩn
#             val_fmax = calculate_fmax_subset(Y_p, Y_t, ia_weights)
#             print(f"Epoch {epoch+1}: Loss={loss_sum/len(train_loader):.4f} | Val F-max={val_fmax:.4f}")
#             torch.save(model.state_dict(), "best_model_c99.pth")
#         else:
#             print(f"Epoch {epoch+1}: Loss={loss_sum/len(train_loader):.4f}")

# if __name__ == "__main__":
#     train_c99_finetune_fixed()

🚀 STARTING C99 FINE-TUNING (FIXED)...
   Loading Vocab C99...
🏥 Starting Model Surgery (C95 -> C99)...
   - Found classifier layers: net.7.weight
   - Copying weights...
   - Copied 6413/15582 terms from C95.
   - Missing keys: 0
   - Unexpected keys: 0
✅ Surgery Complete.
   🔥 Epoch 1: HMC OFF


Ep 1:   0%|          | 0/2318 [00:00<?, ?it/s]/tmp/ipykernel_47/2401601771.py:342: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch 1: Loss=135.9607
   🔥 Epoch 2: HMC OFF


/tmp/ipykernel_47/2123168177.py:167: RuntimeWarning: invalid value encountered in greater_equal
  cut = (p_sub >= tau).astype(int)


Epoch 2: Loss=nan | Val F-max=0.0000
   🔥 Epoch 3: HMC OFF


Ep 3:  82%|████████▏ | 1907/2318 [02:24<00:31, 13.21it/s, loss=nan]